In [1]:
import pandas as pd

# step 1 , load and clean data

In [2]:
df = pd.read_csv('Online Retail.csv')
print(df.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  


In [3]:
print("column names : ", df.columns)

column names :  Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')


In [4]:
# i will check if there is any null value in any columns
print(df.isnull().sum())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [5]:
# we consider null customer id as not completed or rejected invoices so we will remove them

In [6]:
df = df[df['CustomerID'].notna() & (df['CustomerID'] != '')]

In [7]:
# now remove unecessery cols
df = df.drop(["InvoiceDate","UnitPrice","CustomerID","Country"], axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   InvoiceNo    406829 non-null  object
 1   StockCode    406829 non-null  object
 2   Description  406829 non-null  object
 3   Quantity     406829 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 15.5+ MB


In [9]:
# df.describe() , useless in this case

In [10]:
# here i will remove items with 0 or negative quantity
negative_quantity = (df['Quantity'] < 0).sum()
zero_quantity = (df['Quantity'] == 0).sum()
print(f"number of negative quantity values {negative_quantity}, number of zero quantity {zero_quantity}")

number of negative quantity values 8905, number of zero quantity 0


In [11]:
# only keep rows with above zero quantity
# drop description , because some stock numbers are empty
# there is no way we can interpolate or ... them precisely , maybe they are not product but i dont want to lose data , so i will keep
# using them , later i will query data set to get description if available

df = df[df['Quantity'] >= 0]
lookup_df = df.drop(["InvoiceNo","Quantity"], axis=1).drop_duplicates(subset=['StockCode']) # i will use this for finding desc base on stockno
df = df.drop(["Description","Quantity"], axis=1)
print(df.head()) # here step 1 is completed and we have a cleaned data

  InvoiceNo StockCode
0    536365    85123A
1    536365     71053
2    536365    84406B
3    536365    84029G
4    536365    84029E


In [ ]:
def get_descrption(stock_code):
    # filter rows where first column matches stock_code
    rows = lookup_df[lookup_df.iloc[:, 0] == stock_code]
    
    if rows.empty:
        return -1  # stock_code not found
    # description can be nan for one row and non nan for other row
    # iterate over descriptions and return first non nan
    for descr in rows.iloc[:, 1]:
        if pd.notna(descr):
            return descr
    # If all are nan
    return "no description"

    
print(get_descrption("84534B")) # 84534B is an example of an stock_code wiht nan and non nan data

FAIRY CAKE NOTEBOOK A5 SIZE


In [14]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   InvoiceNo  397924 non-null  object
 1   StockCode  397924 non-null  object
dtypes: object(2)
memory usage: 9.1+ MB
None


# step 2 , creating binary matrix

In [ ]:
# here i want to make an new dataframe , rows are invoice no , cols are existance of an item
# binary_df = pd.get_dummies(df['Description'], dtype=int).groupby(df['InvoiceNo']).sum()

'''
bottom part will do one hot encoding ()
pd.get_dummies(df['Description'], dtype=int)
.groupby(df['InvoiceNo']).sum()
it will sum dupplicate invoiceno rows , it will casue the exact thing we want

i ran out of memory :| ...
'''

"\nbottom part will do one hot encoding ()\npd.get_dummies(df['Description'], dtype=int)\n.groupby(df['InvoiceNo']).sum()\nit will sum dupplicate invoiceno rows , it will casue the exact thing we want\n\ni ran out of memory ...\n"

In [24]:
# trying to Sparse[int] , because there will loots of zeros , there are other ways too but this will do the jobS
# binary_df = pd.get_dummies(df['StockCode'], dtype='Sparse[int]').groupby(df['InvoiceNo']).sum()
# this takes for ever!

In [ ]:
df['StockCode'] = df['StockCode'].astype('category')
# as chatgpt says , cross table is exactly what we want in an optimal way :
# Takes two or more categorical inputs (usually Series or arrays).
# Counts how many times each unique pair of values occurs.
# Returns a DataFrame with one factor as rows, the other as columns, and counts as values.
# example :
# data = {
#     'Gender': ['Male', 'Female', 'Female', 'Male', 'Female'],
#     'Preference': ['Football', 'Football', 'Tennis', 'Tennis', 'Football']
# }
# out put look like below :
# | Preference | Football | Tennis |
# | ---------- | -------- | ------ |
# | Gender     |          |        |
# | Female     | 2        | 1      |
# | Male       | 1        | 1      |

binary_df = pd.crosstab(df['InvoiceNo'], df['StockCode'])
binary_df = binary_df.astype('Sparse[int]')
binary_df = (binary_df > 0).astype('Sparse[int]')


In [21]:
binary_df.info()
binary_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 18536 entries, 536365 to 581587
Columns: 3665 entries, 10002 to POST
dtypes: Sparse[int32, 0](3665)
memory usage: 3.1+ MB


StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data_length = binary_df.shape[0]

# step 3 , simulating streaming

In [ ]:
hp = {
    "flow_size" : 1000, # number of rows at each flow is being loaded
    "sketch_size" : 500, # numbre of rows of sketch 
}